<a href="https://colab.research.google.com/github/SEJUNTONY/Data_Science_Class/blob/EunJi/%EC%A1%B0%EB%8B%AC_%EC%9D%80%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리 로드
import pandas as pd
import pandas_datareader.data as pandas_datareader

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('seaborn-whitegrid')

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

import seaborn as sns
import matplotlib.ticker as ticker

import os

plt.style.use('seaborn-whitegrid')
#sns.set_style("white")
%matplotlib inline

import itertools

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# 함수

def outliers_iqr(data) :
    q1, q3 = np.percentile(data, [25,75])
    iqr = q3 - q1
    lower_bound = 0
    upper_bound = 3000

    return np.where((data > upper_bound)|(data<lower_bound))

def get_outlier(df=None, column=None, weight=1.5):
    # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

In [3]:
# 데이터 로드

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 무역 데이터 확인

filename = '/content/drive/MyDrive/공공기관별+시스템유형별+실적(00116)_20220128164342.csv'
vietnam = pd.read_csv(filename)

## 1. 기본정보 확인

EXPOR_NTN_NM, HS_CD, ARRVL_YRMNT, ARRVL_PRT,NM은 **범주형 변수**

VTNM_TRAD_INVCE_AMT는 **수치형 변수**로 파악된다.


컬럼정의서에 따르면 각 컬럼의 의미는 다음과 같다.

1. EXPOR_NTM_NM : 수출국가명
2. HS_CD : HS코드
3. ARRVL_YRMNT
4. VTNM_TRAD_INVCE_AMT : 베트남무역송장금액
5. ARRVL_PRT_NM : 도착항구명

무역데이터에 대한 상향식 접근 방식(Bottom Up Approach)을 고민해볼 것이다.

* 상향식 접근 방식(Bottom Up Approach) : 문제의 정의 자체가 어려운 경우, 데이터를 기반으로 문제의 재정의 및 해결방안을 탐색하고, 이를 지속적으로 개선하는 방식

데이터 분석을 진행하고 싶은 사항은 다음과 같다.

1. HSCODE별 평균 가격 중 높은 순서로 분석

2. 항구별 HSCODE 빈도수가 높은 것

3. 전체 항구의 거래량 파악 : ARRVL_YRMNT 빈도수 파악

In [6]:
vietnam.shape

(30, 13)

5개의 Column이 있다.

In [ ]:
vietnam.info()

In [ ]:
# VTNM_TRAD_INVCE_AMT에 대한 기술통계 Summary
vietnam['VTNM_TRAD_INVCE_AMT'].describe()

기술통계값을 확인해보니, 최소 송장금액이 0인 값이 존재하는 것을 보아, 모델에 오류를 불러일으킬만한 데이터가 존재함을 확인할 수 있다.

In [ ]:
# 송장금액이 0인 값을 제거
A = vietnam[vietnam['VTNM_TRAD_INVCE_AMT']==0].index
vietnam = vietnam.drop(A)

In [ ]:
# 0값 제거 후 VTNM_TRAD_INVCE_AMT에 대한 기술통계 Summary
vietnam['VTNM_TRAD_INVCE_AMT'].describe()

제대로 시행되었음을 확인하였다. 0이 제거된 상태로 히스토그램을 그려보았다.


In [ ]:
# 히스토그램(histogram)
sns.distplot(vietnam['VTNM_TRAD_INVCE_AMT'])

완전히 치우침을 확인할 수 있다. 대체적으로 몰려있는 쪽에 비해 특정 값들이 매우 높아 이런 히스토그램이 그려지는 듯 하다.

In [ ]:
# 왜도 및 첨도
print("Skewness : %f" % vietnam['VTNM_TRAD_INVCE_AMT'].skew())
print("Kurtosis : %f" % vietnam['VTNM_TRAD_INVCE_AMT'].kurt())

왜도 : 자료가 심하게 오른쪽으로 치우쳐있음
첨도 : 심각하게 뾰족한 정규분포의 형태를 보인다.

### (1) HS 코드 분석

평균 가격을 파악하기 이전, HS코드를 분석해보았다.

In [ ]:
vietnam['HS_CD'].unique()

In [ ]:
vietnam['HS_CD'].nunique()

HS코드는 총 2089가지가 있는 것으로 파악된다. 이 값들을 모두 분석할 수는 없기에 상위 값들만을 통해 분석을 진행한다.

In [ ]:
rname = vietnam['HS_CD']
hscode_total = pd.DataFrame(vietnam['HS_CD'].value_counts(), index=rname, )
hscode_total

상위값을 차지하고 있는 품목은 다음과 같다.
1. 39269099
2. 73269099
3. 85332100

거래되는 상위 3개 품목을 확인해본 결과, 플라스틱 소재 등 산업재가 많았다.

https://news.kotra.or.kr/user/globalBbs/kotranews/784/globalBbsDataView.do?setIdx=403&dataIdx=190241

상위 코드 3개에 대한 분석을 진행해보았다.

#### 분석 1 : HS CODE 39269099

In [ ]:
viet_hscd1 = vietnam[(vietnam['HS_CD']==39269099)]
viet_hscd1

날짜 그룹별로 평균을 구해보았다.

In [ ]:
viet_hscd1_lineplot = viet_hscd1.groupby(['ARRVL_YRMNT'], as_index=False).mean()

In [ ]:
amt = "VTNM_TRAD_INVCE_AMT"
arrvl = "ARRVL_YRMNT"
f, ax = plt.subplots(figsize=(24,12))
fig = sns.lineplot(x=arrvl, y=amt, data=viet_hscd1_lineplot)
fig.axis(ymin=0, ymax = 40000);
plt.xticks(rotation=90);

미친 값들이 존재한다.
미친 송장비용을 보여주는 날짜를 그래프를 통해 확인해본 결과, 2019년 5월 10일, 2019년 5월 11일, 2019년 6월 13일, 2019년 8월 3일 정도인 것 같다.
이 날짜의 데이터값을 확인해보았다.

해당 날짜에 대한 데이터행을 삭제하고 lineplot을 다시 그려보았다.

In [ ]:
# 행 제거 위한 행 Index 파악

index1 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2019-05-10'].index
index2 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2019-06-13'].index
index3 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2019-08-03'].index
index4 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2019-05-11'].index
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index1)
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index2)
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index3)
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index4)

In [ ]:
amt = "VTNM_TRAD_INVCE_AMT"
arrvl = "ARRVL_YRMNT"
f, ax = plt.subplots(figsize=(24,12))
fig = sns.lineplot(x=arrvl, y=amt, data=viet_hscd1_lineplot)
fig.axis(ymin=0, ymax = 40000);
plt.xticks(rotation=90);

행 제거 전처리 후 데이터를 통해 시계열 분석을 진행해보았다.

In [ ]:
viet_hscd1_lineplot2 = viet_hscd1_lineplot.drop(['HS_CD'], axis=1)

In [ ]:
viet_hscd1_lineplot2['ARRVL_YRMNT'] = pd.to_datetime(viet_hscd1_lineplot2['ARRVL_YRMNT'])
viet_hscd1_lineplot2 = viet_hscd1_lineplot2.set_index('ARRVL_YRMNT')
viet_hscd1_lineplot2.head(10)

In [ ]:
fig = viet_hscd1_lineplot2.plot()

2020년 데이터가 없는 것으로 보아, 2020년 이후 데이터는 제거하는게 타당해보였다.

In [ ]:
index1 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2019-05-10'].index
index2 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2019-06-13'].index
index3 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2019-08-03'].index
index4 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2019-05-11'].index
index5 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2021-02-22'].index
index6 = viet_hscd1_lineplot[viet_hscd1_lineplot['ARRVL_YRMNT']=='2021-02-23'].index
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index1)
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index2)
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index3)
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index4)
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index5)
viet_hscd1_lineplot = viet_hscd1_lineplot.drop(index6)

viet_hscd1_lineplot2 = viet_hscd1_lineplot.drop(['HS_CD'], axis=1)
viet_hscd1_lineplot2['ARRVL_YRMNT'] = pd.to_datetime(viet_hscd1_lineplot2['ARRVL_YRMNT'])
viet_hscd1_lineplot2 = viet_hscd1_lineplot2.set_index('ARRVL_YRMNT')

In [ ]:
fig = viet_hscd1_lineplot2.plot()

데이터 수가 얼마 없지만 일말의 가능성을 염두하고 ARIMA 분석을 진행하였다.

In [ ]:
data = viet_hscd1_lineplot2
data.head(10)

계략적인 모델을 구축해보았다.

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

In [ ]:
# ACF, PACF plot

fig, ax = plt.subplots(1,2,figsize=(10,10))
fig.suptitle('Raw Data')
sm.graphics.tsa.plot_acf(train_data.values.squeeze(), lags=30, ax=ax[0])
sm.graphics.tsa.plot_pacf(train_data.values.squeeze(), lags=30, ax = ax[1])

파란색 범위 안에 값이 들어가있는 것을 보면, autocorrelation이 있다고 판단할 수 있다.
이후 차분을 진행해보았다.

In [ ]:
diff_train_data = train_data.copy()
diff_train_data = diff_train_data['VTNM_TRAD_INVCE_AMT'].diff()
diff_train_data = diff_train_data.dropna()
print('#### Raw Data ####')
print(train_data)
print('#### Differenced Data ###')
print(diff_train_data)

In [ ]:
# Differenced Data Plot

plt.figure(figsize=(12,8))
plt.subplot(211)
plt.plot(train_data['VTNM_TRAD_INVCE_AMT'])
plt.legend(['Raw Data (Reasonably Stationary)'])
plt.subplot(212)
plt.plot(diff_train_data, 'orange')
plt.legend(['Differenced Data'])
plt.show()

2019년 4월과 5월 사이 값이 확 뛰는 구간이 있었기 때문에 differenced 후 그 부분이 완벽하게 보정되는 느낌은 아니다. 실제 차분한 값을 기반으로 autocorrelation을 확인해보았다.

In [ ]:
# ACF, PACF plot

fig, ax = plt.subplots(1,2,figsize=(10,5))
fig.suptitle('Differenced Data')
sm.graphics.tsa.plot_acf(diff_train_data.values.squeeze(), lags=30, ax=ax[0])
sm.graphics.tsa.plot_pacf(diff_train_data.values.squeeze(), lags=30, ax=ax[1])

현재 확인되는 그래프로는 p,d,q를 파악하기 어려워보인다.

In [ ]:
# Diagnosis Check : ARIMA

# Parameter Search

print('Examples of Parameter Combinations for Seasonal ARIMA...')
p = range(0,3)
d = range(1,2)
q = range(0,3)
pdq = list(itertools.product(p,d,q))

aic=[]
for i in pdq :
    model = ARIMA(train_data.values, order=(i))
    model_fit = model.fit()
    print(f'ARIMA : {i} >> AIC : {round(model_fit.aic,2)}')
    aic.append(round(model_fit.aic,2))

상위 코드 3개에 대한 분석을 진행해보았다.

#### 분석 2 : HS CODE 73269099

In [ ]:
viet_hscd2 = vietnam[(vietnam['HS_CD']==73269099)]
viet_hscd2

In [ ]:
viet_hscd1_lineplot2 = viet_hscd2.groupby(['ARRVL_YRMNT'], as_index=False).mean()

In [ ]:
amt = "VTNM_TRAD_INVCE_AMT"
arrvl = "ARRVL_YRMNT"
f, ax = plt.subplots(figsize=(24,12))
fig = sns.lineplot(x=arrvl, y=amt, data=viet_hscd1_lineplot2)
fig.axis(ymin=0, ymax = 40000);
plt.xticks(rotation=90);

--> 월별로 묶어서 평균내봐서 lineplot 그려보기

--> 남부가 더 물동량이 높음.

남부 : VNCLI, VNSGN
북부 : VNDVU, VAHAN, VNCXP

## 데이터를 통해 정의한 문제

1. 남부 송장비용이 높은 지, 북부가 높은 지
2. 남부가 경제도시이기에 거래량이 많음
3. HS CODE 분석 : 남부 vs 북부
4. 거래량 시계열 데이터 vs 송장비용 시계열 데이터


### 1. 남부항구의 송장비용이 높은가? 북부항구의 송장비용이 높은가?

물동량이 많은 순서로 정렬 후, 남부 항구는 VNCLI, VNSGN로, 북부항구는 VNDVU, VAHAN, VNCXP로 정의하였다.

In [ ]:
A = (vietnam.ARRVL_PRT_NM == 'VNCLI') | (vietnam.ARRVL_PRT_NM == 'VNSGN')
viet_south = vietnam[A]
viet_south
B = (vietnam.ARRVL_PRT_NM == 'VNDVU') | (vietnam.ARRVL_PRT_NM == 'VNHAN') | (vietnam.ARRVL_PRT_NM == 'VNCXP')
viet_north = vietnam[B]
viet_north
C = (vietnam.ARRVL_PRT_NM == 'VNDVU') | (vietnam.ARRVL_PRT_NM == 'VNCXP')
viet_north2 = vietnam[C]
viet_north2

In [ ]:
viet_south_sum = viet_south.groupby(['ARRVL_PRT_NM'], as_index=False).sum()
sns.barplot(data=viet_south_sum, x="ARRVL_PRT_NM", y="VTNM_TRAD_INVCE_AMT")

In [ ]:
viet_north_sum = viet_north.groupby(['ARRVL_PRT_NM'], as_index=False).sum()
f, ax = plt.subplots(figsize=(24,12))
fig = sns.barplot(data=viet_north_sum, x="ARRVL_PRT_NM", y="VTNM_TRAD_INVCE_AMT")
fig.axis(ymin=0, ymax = 3.5e+08);

In [ ]:
viet_north_sum2 = viet_north2.groupby(['ARRVL_PRT_NM'], as_index=False).sum()
f, ax = plt.subplots(figsize=(24,12))
fig = sns.barplot(data=viet_north_sum2, x="ARRVL_PRT_NM", y="VTNM_TRAD_INVCE_AMT")